In [1]:
%load_ext supriya.ext.ipython
import supriya

## Realtime Server Node Tree Model

In [20]:
synth_a = supriya.Synth(frequency=99)
synth_b = supriya.Synth(frequency=666)
synth_c = supriya.Synth(frequency=3333)
outer_group = supriya.Group()
inner_group = supriya.Group()
outer_group.extend([synth_a, inner_group, synth_c])
inner_group.append(synth_b)

In [21]:
# groups know about their children
for node in outer_group.children:
    print(repr(node))

<- Synth: ???>
<- Group: ???>
<- Synth: ???>


In [22]:
# and nodes know about their parents
for node in synth_b.parentage:
    print(repr(node))

<- Synth: ???>
<- Group: ???>
<- Group: ???>


In [23]:
server = supriya.Server()
server.boot()

<Server: udp://127.0.0.1:57751, 8i8o>

In [24]:
# we can query the node tree via /g_queryTree requests
print(server)

NODE TREE 0 group
    1 group


In [7]:
# allocating a group also recursively allocates its children
outer_group.allocate()
print(server)

NODE TREE 0 group
    1 group
        1000 group
            1001 default
                out: 0.0, amplitude: 0.1, frequency: 99.0, gate: 1.0, pan: 0.5
            1002 group
                1003 default
                    out: 0.0, amplitude: 0.1, frequency: 666.0, gate: 1.0, pan: 0.5
            1004 default
                out: 0.0, amplitude: 0.1, frequency: 3333.0, gate: 1.0, pan: 0.5


TODO: Are SynthDef's marked as allocated client-side when allocated during node allocation?

In [30]:
synth_d = supriya.Synth(synthdef=supriya.assets.synthdefs.pad).allocate()

TIMED OUT: QuitRequest()


In [29]:
# synthdefs are also automatically allocated
synth_d = supriya.Synth(synthdef=supriya.assets.synthdefs.pad)
with server.osc_io.capture() as capture:
    inner_group.extend([synth_d, synth_a])
    
for message in capture.sent_messages:
    print(repr(message))

In [14]:
for node in outer_group.children:
    print(repr(node))

<+ Group: 1002>
<+ Synth: 1004>


In [15]:
# the default group
for node in synth_b.parentage:
    print(repr(node))

<+ Synth: 1003>
<+ Group: 1002>
<+ Group: 1000>
<+ Group: 1>
<+ RootNode: 0>


In [16]:
outer_group.free()

<- Group: ???>

In [17]:
print(server)

NODE TREE 0 group
    1 group
        1006 pad
            out: 0.0, amplitude: 0.1, filter_frequency: 1500.0, frequency: 440.0, gate: 1.0, pan: 0.0


In [18]:
print(outer_group)

??? group
    ??? group
        ??? default
            amplitude: 0.1, frequency: 666.0, gate: 1.0, out: 0.0, pan: 0.5
        ??? pad
            amplitude: 0.1, filter_frequency: 1500.0, frequency: 440.0, gate: 1.0, out: 0.0, pan: 0.0
        ??? pad
            amplitude: 0.1, filter_frequency: 1500.0, frequency: 440.0, gate: 1.0, out: 0.0, pan: 0.0
        ??? pad
            amplitude: 0.1, filter_frequency: 1500.0, frequency: 440.0, gate: 1.0, out: 0.0, pan: 0.0
        ??? default
            amplitude: 0.1, frequency: 99.0, gate: 1.0, out: 0.0, pan: 0.5
    ??? default
        amplitude: 0.1, frequency: 3333.0, gate: 1.0, out: 0.0, pan: 0.5


In [19]:
server.quit()

<Server: offline>

## OSC Command Aggregation

## Requests and Responses

## SynthDef Builders

SynthDefs are built via context managers, not via the namespaces of functions

SynthDefs do not need to be named; Supriya uses hashing to generate unique names

## Graphviz Everything

## SynthDef Factories

## Non-realtime Session Model

## Non-realtime Session "Renderables"

## NRT Dependency Tree (turtles all the way down)

## (N)RT Patterns

## CLI Tooling and NRT Projects